In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="uZ059uALtKNidvB7BeHR")
project = rf.workspace("log-wildfire").project("forest-fire-panel-tile")
version = project.version(1)
dataset = version.download("coco-segmentation")

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Forest-Fire-Panel-Tile-1 in coco-segmentation:: 100%|██████████| 10/10 [00:00<00:00, 63.27it/s]


In [2]:
from pathlib import Path
import json
import cv2
import numpy as np
from pycocotools.coco import COCO

def coco_to_mmsegmentation(
    annotations_file: str, output_annotations_file: str, output_masks_dir: str
):
    """Convert json in [segmentation format](https://gradiant.github.io/ai-dataset-template/supported_tasks/#segmentation) to txt in [mmsegmentation format](https://mmsegmentation.readthedocs.io/en/latest/tutorials/new_dataset.html#reorganize-dataset-to-existing-format).

    Args:
        annotations_file:
            path to json in [segmentation format](https://gradiant.github.io/ai-dataset-template/supported_tasks/#segmentation)
        output_annotations_file:
            path to write the txt in [mmsegmentation format](https://mmsegmentation.readthedocs.io/en/latest/tutorials/customize_datasets.html#customize-datasets-by-reorganizing-data)
        output_masks_dir:
            path where the masks generated from the annotations will be saved to.
            A single `{file_name}.png` mask will be generated for each image.
    """

    Path(output_annotations_file).parent.mkdir(parents=True, exist_ok=True)
    Path(output_masks_dir).mkdir(parents=True, exist_ok=True)

    print(f"Loading annotations form {annotations_file}")
    annotations = json.load(open(annotations_file))

    print(f"Saving annotations to {output_annotations_file}")
    with open(output_annotations_file, "w") as f:
        for image in annotations["images"]:
            filename = Path(image["file_name"]).parent / Path(image["file_name"]).stem
            f.write(str(filename))
            f.write("\n")

    print(f"Saving masks to {output_masks_dir}")
    coco_annotations = COCO(annotations_file)
    for image_id, image_data in coco_annotations.imgs.items():
        filename = image_data["file_name"]

        anns_ids = coco_annotations.getAnnIds(imgIds=image_id)
        image_annotations = coco_annotations.loadAnns(anns_ids)

        print(f"Creating output mask for {filename}")

        output_mask = np.zeros(
            (image_data["height"], image_data["width"]), dtype=np.uint8
        )
        for image_annotation in image_annotations:
            category_id = image_annotation["category_id"]
            try:
                category_mask = coco_annotations.annToMask(image_annotation)
            except Exception as e:
                print(e)
                print(f"Skipping {image_annotation}")
                continue
            category_mask *= category_id
            category_mask *= output_mask == 0
            output_mask += category_mask

        output_filename = Path(output_masks_dir) / Path(filename).with_suffix(".png")
        output_filename.parent.mkdir(parents=True, exist_ok=True)

        print(f"Writting mask to {output_filename}")
        cv2.imwrite(str(output_filename), output_mask)

In [3]:
!mkdir Forest-Fire-Panel-Tile-1/labels/ Forest-Fire-Panel-Tile-1/splits/ Forest-Fire-Panel-Tile-1/images/

In [4]:
coco_to_mmsegmentation("Forest-Fire-Panel-Tile-1/train/_annotations.coco.json", "Forest-Fire-Panel-Tile-1/splits/train.txt", "Forest-Fire-Panel-Tile-1/labels/")

Loading annotations form Forest-Fire-Panel-Tile-1/train/_annotations.coco.json
Saving annotations to Forest-Fire-Panel-Tile-1/splits/train.txt
Saving masks to Forest-Fire-Panel-Tile-1/labels/
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Creating output mask for 0_jpg.rf.212efdea0f69eaeda7ba253e3c0bac33.jpg
Writting mask to Forest-Fire-Panel-Tile-1/labels/0_jpg.rf.212efdea0f69eaeda7ba253e3c0bac33.png
Creating output mask for 0_jpg.rf.5afda1725ca7f119511f72c1004f8920.jpg
Writting mask to Forest-Fire-Panel-Tile-1/labels/0_jpg.rf.5afda1725ca7f119511f72c1004f8920.png
Creating output mask for 0_jpg.rf.27a7915f68c1125285823e6d0bb66a7c.jpg
Writting mask to Forest-Fire-Panel-Tile-1/labels/0_jpg.rf.27a7915f68c1125285823e6d0bb66a7c.png
Creating output mask for 0_jpg.rf.785715033271869e823a1562cd458020.jpg
Writting mask to Forest-Fire-Panel-Tile-1/labels/0_jpg.rf.785715033271869e823a1562cd458020.png
Creating output mask for 0_jpg.rf.cc0224a31d42b38e51f168e7429

In [6]:
# define dataset root and directory for images and annotations
data_root = 'Forest-Fire-Panel-Tile-1'
img_dir = 'train'
ann_dir = 'labels'
new_ann_dir = 'rgb_labels'
# define class and palette for better visualization
classes = ('BG', 'Dead Tree')
palette = [[0, 0, 0], [10, 20, 255]]

In [9]:
import os
import os.path as osp
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# convert dataset annotation to semantic segmentation map
# for file in mmengine.scandir(osp.join(data_root, ann_dir), suffix='.regions.txt'):
#   seg_map = np.loadtxt(osp.join(data_root, ann_dir, file)).astype(np.uint8)
#   seg_img = Image.fromarray(seg_map).convert('P')
#   seg_img.putpalette(np.array(palette, dtype=np.uint8))
#   seg_img.save(osp.join(data_root, ann_dir, file.replace('.regions.txt',
#                                                          '.png')))
for file in os.listdir(osp.join(data_root, ann_dir)):
    if file.endswith('.png'):
        seg_map = Image.open(osp.join(data_root, ann_dir, file))
        seg_img = seg_map.convert('P')
        seg_img.putpalette(np.array(palette, dtype=np.uint8))
        seg_img.save(osp.join(data_root, new_ann_dir, file))